# 데이터 셋 준비

- 구글 드라이브를 마운트하는 코드이다.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## 드라이브 내의 데이터셋 폴더에서 작동시키는 경우

In [ ]:
dataset_dir = "./drive/MyDrive/DCC_dataset" # 드라이브에서 작동

## 드라이브 내의 압축파일에서 코랩 로컬로 압축 해제하여 사용하는 경우

- 데이터셋 파일의 압축을 푸는 코드이다.
  - 압축 파일은 구글 드라이브의 루트에서 이름은 *2024 데이터 크리에이터 캠프 대학부 데이터셋.zip*로 존재하고 있어야 한다.

In [ ]:
!unzip -qq "./drive/MyDrive/2024 데이터 크리에이터 캠프 대학부 데이터셋.zip" -d "./DCC_datasets/"

In [ ]:
dataset_dir = "./DCC_datasets" # 코랩 로컬에서 작동

# Mission 2. Label 데이터 가공
---

## 2-1. 유효한 레이블 파일 유저ID 기준 통계표 제작
> **실제 대응되는 이미지가 존재하는 트레이닝, 검증 레이블 데이터만을 골라 통계표를 작성하라.**

### 2-1.1. 트레이닝 데이터와 레이블을 검증하고 통계표 제작

- Training Image와 Training Label의 유효성 검사를 진행

In [ ]:
import os
import re
import pandas as pd
from collections import Counter

# 트레이닝 이미지 및 레이블 파일 경로
training_image_dir = f'{dataset_dir}/training_image/'  # 트레이닝 이미지 파일 경로
training_label_dir = f'{dataset_dir}/training_label/'  # 트레이닝 레이블 파일 경로

# 트레이닝 이미지와 레이블 파일 불러오기
image_files = os.listdir(training_image_dir)
label_files = os.listdir(training_label_dir)

# 이미지와 레이블 파일의 공통 key 추출을 위한 정규표현식 패턴
image_pattern = re.compile(r'(W|T)_(\d+)_(\d+)_(\w+)_(W|M)\.jpg')
label_pattern = re.compile(r'(W|T)_(\d+)_(\d+)_(\w+)_(W|M)_\d+\.json')

# 유효한 트레이닝 레이블의 파일명을 저장할 리스트 : 2-2에 사용된다.
training_valid_labels = []

# 트레이닝 이미지 파일에서 key를 추출 (유효성 검사 진행)
image_keys = set()
for image_file in image_files:
    match = image_pattern.match(image_file)
    if match:
        image_key = f"{match.group(1)}_{match.group(2)}_{match.group(3)}_{match.group(4)}_{match.group(5)}"
        image_keys.add(image_key)

# 트레이닝 레이블 파일에서 key를 추출하고, 유효성 검사
label_counts = Counter()

for label_file in label_files:
    match = label_pattern.match(label_file)
    if match:
        # 레이블 파일에서 추출된 key와 트레이닝 이미지 키를 비교하여 유효성 검사
        label_key = f"{match.group(1)}_{match.group(2)}_{match.group(3)}_{match.group(4)}_{match.group(5)}"
        if label_key in image_keys:  # 트레이닝 이미지에 존재하는 경우 유효한 레이블로 처리
            training_valid_labels.append(label_file)
            label_counts[label_key] += 1

# 레이블 통계를 성별, 스타일 기준으로 집계
data = []

for label_key, count in label_counts.items():
    parts = label_key.split('_')
    gender = '여성' if parts[4] == 'W' else '남성'
    style = parts[3]
    data.append([gender, style, count])

# 데이터프레임 생성
df = pd.DataFrame(data, columns=['성별', '스타일', '레이블 수'])

# 성별과 스타일 기준으로 이미지 수 합계 계산
df_grouped = df.groupby(['성별', '스타일']).sum().reset_index()

# 결과 출력
print(df_grouped)

    성별             스타일  레이블 수
0   남성            bold   1007
1   남성          hiphop   1070
2   남성          hippie   1502
3   남성             ivy   1608
4   남성     metrosexual   1045
5   남성            mods   1458
6   남성        normcore    644
7   남성  sportivecasual    845
8   여성      athleisure    378
9   여성   bodyconscious    455
10  여성        cityglam    231
11  여성         classic    360
12  여성           disco    130
13  여성         ecology    223
14  여성        feminine    719
15  여성      genderless    146
16  여성          grunge     90
17  여성          hiphop    138
18  여성          hippie    308
19  여성          kitsch    261
20  여성        lingerie    156
21  여성          lounge     82
22  여성        military    109
23  여성         minimal    643
24  여성        normcore    278
25  여성        oriental    282
26  여성          popart    193
27  여성       powersuit    531
28  여성            punk    218
29  여성           space    171
30  여성  sportivecasual    815


In [ ]:
print(f'valid labels count: {len(training_valid_labels)}, all training labels count: {len(label_files)}') # 유효한 훈련 레이블 데이터의 수

valid labels count: 16096, all training labels count: 211346


- 시각화

In [ ]:
import plotly.graph_objects as go

# 성별에 따라 색상을 지정하는 함수
def get_fill_colors(genders):
    fill_colors = []
    for gender in genders:
        if gender == '남성':
            fill_colors.append('rgb(203, 230, 211)')
        elif gender == '여성':
            fill_colors.append('rgb(225, 233, 246)')
        else:
            fill_colors.append('white')
    return fill_colors

# Plotly 테이블 생성
fig = go.Figure(data=[go.Table(
    columnwidth=[80, 100, 80],  # 각 열의 너비를 설정
    header=dict(values=['성별', '스타일', '레이블 수'],
                fill_color='rgb(33, 107, 171)',
                font_color='white',
                align='center',
                font_size=27,
                height=40),

    cells=dict(values=[df_grouped['성별'], df_grouped['스타일'], df_grouped['레이블 수']],
               fill_color=[get_fill_colors(df_grouped['성별']), 'white', 'white'],  # 성별에 따른 배경 색상
               line_color='rgb(33, 107, 171)',
               font_color='rgb(33, 107, 171)',
               font_size=20,
               align='center',
               height=30))])

# 제목 추가
fig.update_layout(
    title={
        'text': "성별과 스타일에 따른 유효한 트레이닝 레이블 수",
        'y':0.9,
        'x':0.5,
        'xanchor': 'center',
        'yanchor': 'top',
        'font_color': 'rgb(33, 107, 171)',
        'font_size': 30
    }
)

# 테이블 출력
fig.show()

### 2-1.2. 검증 데이터와 레이블을 검증하고 통계표 제작

- Validation Image와 Validation Label의 유효성 검사를 진행

In [ ]:
import os
import re
import pandas as pd
from collections import Counter

# 검증 이미지 및 레이블 파일 경로
validation_image_dir = f'{dataset_dir}/validation_image/'  # 검증 이미지 파일 경로
valdiation_label_dir = f'{dataset_dir}/validation_label/'  # 검증 레이블 파일 경로

# 검증 이미지와 레이블 파일 불러오기
image_files = os.listdir(validation_image_dir)
label_files = os.listdir(valdiation_label_dir)

# 이미지와 레이블 파일의 공통 key 추출을 위한 정규표현식 패턴
image_pattern = re.compile(r'(W|T)_(\d+)_(\d+)_(\w+)_(W|M)\.jpg')
label_pattern = re.compile(r'(W|T)_(\d+)_(\d+)_(\w+)_(W|M)_\d+\.json')

# 유효한 검증 레이블의 파일명을 저장할 리스트
validation_valid_labels = []

# 검증 이미지 파일에서 key를 추출 (유효성 검사 진행)
image_keys = set()
for image_file in image_files:
    match = image_pattern.match(image_file)
    if match:
        image_key = f"{match.group(1)}_{match.group(2)}_{match.group(3)}_{match.group(4)}_{match.group(5)}"
        image_keys.add(image_key)

# 검증 레이블 파일에서 key를 추출하고, 유효성 검사
label_counts = Counter()

for label_file in label_files:
    match = label_pattern.match(label_file)
    if match:
        # 레이블 파일에서 추출된 key와 검증 이미지 키를 비교하여 유효성 검사
        label_key = f"{match.group(1)}_{match.group(2)}_{match.group(3)}_{match.group(4)}_{match.group(5)}"
        if label_key in image_keys:  # 검증 이미지에 존재하는 경우 유효한 레이블로 처리
            validation_valid_labels.append(label_file)
            label_counts[label_key] += 1

# 레이블 통계를 성별, 스타일 기준으로 집계
data = []

for label_key, count in label_counts.items():
    parts = label_key.split('_')
    gender = '여성' if parts[4] == 'W' else '남성'
    style = parts[3]
    data.append([gender, style, count])

# 데이터프레임 생성
df = pd.DataFrame(data, columns=['성별', '스타일', '이미지 수'])

# 성별과 스타일 기준으로 이미지 수 합계 계산
df_grouped = df.groupby(['성별', '스타일']).sum().reset_index()

# 결과 출력
print(df_grouped)

    성별             스타일  이미지 수
0   남성            bold    215
1   남성          hiphop    256
2   남성          hippie    474
3   남성             ivy    537
4   남성     metrosexual    224
5   남성            mods    437
6   남성        normcore     89
7   남성  sportivecasual    148
8   여성      athleisure     80
9   여성   bodyconscious    111
10  여성        cityglam     61
11  여성         classic    102
12  여성           disco     31
13  여성         ecology     65
14  여성        feminine    208
15  여성      genderless     24
16  여성          grunge     29
17  여성          hiphop     23
18  여성          hippie     46
19  여성          kitsch     61
20  여성        lingerie     15
21  여성          lounge     16
22  여성        military     32
23  여성         minimal    164
24  여성        normcore     39
25  여성        oriental     67
26  여성          popart     38
27  여성       powersuit    151
28  여성            punk     38
29  여성           space     70
30  여성  sportivecasual    254


In [ ]:
print(f'valid validation labels count: {len(validation_valid_labels)}, all validation labels count: {len(label_files)}') # 유효한 훈련 레이블 데이터의 수

valid validation labels count: 4105, all validation labels count: 36383


- 시각화

In [ ]:
import plotly.graph_objects as go

# 성별에 따라 색상을 지정하는 함수
def get_fill_colors(genders):
    fill_colors = []
    for gender in genders:
        if gender == '남성':
            fill_colors.append('rgb(203, 230, 211)')
        elif gender == '여성':
            fill_colors.append('rgb(225, 233, 246)')
        else:
            fill_colors.append('white')
    return fill_colors

# Plotly 테이블 생성
fig = go.Figure(data=[go.Table(
    columnwidth=[80, 100, 80],  # 각 열의 너비를 설정
    header=dict(values=['성별', '스타일', '레이블 수'],
                fill_color='rgb(33, 107, 171)',
                font_color='white',
                align='center',
                font_size=27,
                height=40),

    cells=dict(values=[df_grouped['성별'], df_grouped['스타일'], df_grouped['이미지 수']],
               fill_color=[get_fill_colors(df_grouped['성별']), 'white', 'white'],  # 성별에 따른 배경 색상
               line_color='rgb(33, 107, 171)',
               font_color='rgb(33, 107, 171)',
               font_size=20,
               align='center',
               height=30))])

# 제목 추가
fig.update_layout(
    title={
        'text': "성별과 스타일에 따른 유효한 검증 레이블 데이터 수",
        'y':0.9,
        'x':0.5,
        'xanchor': 'center',
        'yanchor': 'top',
        'font_color': 'rgb(33, 107, 171)',
        'font_size': 30
    }
)

# 테이블 출력
fig.show()

## 2-2. 유효한 레이블 데이터를 사용한 100명 응답자의 스타일 선호표 제작


### 2-2.1. 전체 응답자의 스타일 선호표 제작

- 앞서 정의했던 유효한 레이블 파일명 리스트를 사용하여 응답자 ID, 선호 여부, 이미지 파일 명을 데이터 프레임의 형태로 정리하는 코드

- 시각화 코드

In [ ]:
import os
import json
import pandas as pd

# 파일명들이 담겨 있는 리스트에서 R_ID, 선호 여부, 파일명을 추출하는 함수
def extract_values(label_file_list, group_):
    # 응답자 ID와 스타일 선호 정보 수집을 위한 리스트
    extracted_data = []
    for label_file in label_file_list:
        group = group_
        label_file = f'{dataset_dir}/{group}_label/{label_file}'
        with open(label_file, 'r', encoding='utf-8') as f:
            label_data = json.load(f)
            # 응답자 ID 추출
            respondent_id = label_data['user']['R_id']

            # 스타일 선호 여부 추출
            style_preference = label_data['item']['survey']['Q5']

            # 이미지 파일명 추출
            image_file_name = label_data['item']['imgName']

            # 선호 여부에 따라 데이터 추가
            if style_preference == 2:  # 2는 선호
                preference_type = '선호'
            elif style_preference == 1:  # 1은 비선호
                preference_type = '비선호'
            else:
                continue  # 1 또는 2가 아닌 경우 무시`
            # 데이터 리스트에 추가
            extracted_data.append([respondent_id, group, preference_type, image_file_name])
    return extracted_data

data = extract_values(training_valid_labels, "training") + extract_values(validation_valid_labels, "validation")

# 데이터프레임으로 변환
df = pd.DataFrame(data, columns=['응답자ID', '그룹', '선호여부', '이미지파일'])

# 각 응답자의 선호/비선호 스타일 정보를 그룹별로 정리
df_preference = df.pivot_table(index='응답자ID', columns=['그룹', '선호여부'], values='이미지파일', aggfunc=lambda x: ', '.join(x))

# 열 이름 정리
df_preference.columns = ['_'.join(col).strip() for col in df_preference.columns.values]

# 결과 출력
print(df_preference)

                                            training_비선호  \
응답자ID                                                      
12                                                   NaN   
25                          W_12740_00_metrosexual_M.jpg   
26                             W_18990_50_feminine_W.jpg   
27     W_15212_60_mods_M.jpg, W_07347_90_hiphop_M.jpg...   
30                                 W_49287_70_punk_W.jpg   
...                                                  ...   
68396  W_17460_00_metrosexual_M.jpg, W_24537_70_hippi...   
68398                              W_25360_80_bold_M.jpg   
68729                                                NaN   
68743                    T_01883_10_sportivecasual_M.jpg   
68747                                                NaN   

                                             training_선호  \
응답자ID                                                      
12                              W_03412_50_classic_W.jpg   
25                                     

In [ ]:
import plotly.graph_objects as go
import pandas as pd

# 비어있지 않은 셀의 색상을 초록색으로 채우는 함수
def get_fill_colors(values):
    fill_colors = []
    for v in values:
        if v == '비어있음':  # '비어있음' 문자열을 white로 채색
            fill_colors.append('white')
        else:
            fill_colors.append('rgb(203, 230, 211)')
    return fill_colors

# NaN 값을 "비어있음"으로 변경
df_preference_filled = df_preference.fillna('비어있음')

# 테이블 생성
fig = go.Figure(data=[go.Table(
    columnwidth=[40, 100, 100, 100, 100],
    header=dict(values=['응답자 ID', '훈련: 선호', '훈련: 비선호', '검증: 선호', '검증: 비선호'],
                fill_color='rgb(33, 107, 171)',
                font_color='white',
                align='center',
                font_size=25,
                height=40),
    cells=dict(values=[df_preference_filled.index.tolist(),
                       df_preference_filled['training_선호'],
                       df_preference_filled['training_비선호'],
                       df_preference_filled['validation_선호'],
                       df_preference_filled['validation_비선호']],
               fill_color=['white',
                           get_fill_colors(df_preference_filled['training_선호']),
                           get_fill_colors(df_preference_filled['training_비선호']),
                           get_fill_colors(df_preference_filled['validation_선호']),
                           get_fill_colors(df_preference_filled['validation_비선호'])],
               line_color='rgb(33, 107, 171)',
               font_color='rgb(33, 107, 171)',
               font_size=20,
               align='center',
               height=50))
])

# 제목 추가
fig.update_layout(
    title={
        'text': "응답자별 선호 및 비선호 이미지 전체 목록",
        'y': 0.9,
        'x': 0.5,
        'xanchor': 'center',
        'yanchor': 'top',
        'font_color': 'rgb(33, 107, 171)',
        'font_size': 30
    }
)

# 테이블 출력
fig.show()


### 2-2.2. 상위 100명 산출하기

- 가장 가치있는 100명 응답자의 스타일 선호표를 작성하려고 함
  - 그래서 다음의 기준을 통해 내림차순으로 선별함
    - 1. 모든 열이 가득 차 있는가?
    - 2. 총 리뷰한 이미지 수는 얼마인가?
- 아래는 그것을 구현하여 상위 100명의 데이터 프레임을 제작하는 코드임

In [ ]:
# 응답자의 실제 데이터 존재 여부 확인 함수
def has_valid_vectors(row):
    # 각 셀이 NaN이 아니고 빈 문자열이 아닌지 확인하여 유효성 판단
    return not (row.isna().any() or (row == '').any())

# 1. 모든 열이 가득 차 있고 NaN이 아닌 행들만 선택
df_filled = df_preference.dropna(how='any')  # NaN이 있는 행 제외
df_filled = df_filled[~(df_filled == '').any(axis=1)]  # 빈 문자열이 있는 행 제외

# 2. 데이터가 없는 응답자를 제외 (has_valid_vectors 함수 사용)
df_filled = df_filled[df_filled.apply(has_valid_vectors, axis=1)]

# 3. 각 셀의 이미지 파일명 수를 계산하는 함수
def count_images(cell):
    if isinstance(cell, str):
        return len(cell.split(', '))
    else:
        return 0

# 4. 각 행마다 총 이미지 파일명 수를 계산
df_filled['총_이미지_수'] = df_filled.apply(
    lambda row: sum([count_images(row[col]) for col in df_filled.columns]),
    axis=1
)

# 5. 총 이미지 수를 기준으로 내림차순 정렬하여 상위 100개 행 선택
top_100_rows = df_filled.sort_values(by='총_이미지_수', ascending=False).head(100)

# 결과 출력
print(top_100_rows)


                                            training_비선호  \
응답자ID                                                      
64747  W_47169_70_hippie_W.jpg, W_02498_50_feminine_W...   
64346  W_33015_19_normcore_M.jpg, W_15769_00_metrosex...   
63405  W_17108_19_normcore_M.jpg, W_12443_90_hiphop_M...   
65139  W_26315_19_normcore_M.jpg, W_24629_00_metrosex...   
64561  W_49287_70_punk_W.jpg, W_02232_70_hippie_W.jpg...   
...                                                  ...   
63927  W_17191_19_normcore_M.jpg, W_17040_19_normcore...   
63989  W_15565_70_hippie_M.jpg, W_04324_90_hiphop_M.j...   
50357  W_07324_50_ivy_M.jpg, W_02684_60_mods_M.jpg, W...   
28828  W_15389_60_mods_M.jpg, W_09771_60_mods_M.jpg, ...   
18730  W_12583_70_hippie_M.jpg, W_15795_70_hippie_M.j...   

                                             training_선호  \
응답자ID                                                      
64747  W_21483_19_genderless_W.jpg, W_30434_60_minima...   
64346  W_49510_00_metrosexual_M.jpg, W_

- 시각화

In [ ]:
# 테이블 생성
# 비어있지 않은 셀의 색상을 초록색으로 채우는 함수
def get_fill_colors(values):
    fill_colors = []
    for v in values:
      if v == '':
        fill_colors.append('white')
      else:
        fill_colors.append('rgb(203, 230, 211)')
    return fill_colors

fig = go.Figure(data=[go.Table(
    columnwidth=[40, 100, 100, 100, 100],
    header=dict(values=['응답자 ID', '훈련: 선호', '훈련: 비선호', '검증: 선호', '검증: 비선호'],
                fill_color='rgb(33, 107, 171)',
                font_color='white',
                align='center',
                font_size=25,
                height=40),
    cells=dict(values=[top_100_rows.index.tolist(),
                       top_100_rows['training_선호'],
                       top_100_rows['training_비선호'],
                       top_100_rows['validation_선호'],
                       top_100_rows['validation_비선호']],
               fill_color=['white',
                           get_fill_colors(top_100_rows['training_선호']),
                           get_fill_colors(top_100_rows['training_비선호']),
                           get_fill_colors(top_100_rows['validation_선호']),
                           get_fill_colors(top_100_rows['validation_비선호'])],
               line_color='rgb(33, 107, 171)',
               font_color='rgb(33, 107, 171)',
               font_size=20,
               align='center',
               height=50))
])

# 제목 추가
fig.update_layout(
    title={
        'text': "상위 100개 응답자의 선호 및 비선호 이미지 목록",
        'y': 0.9,
        'x': 0.5,
        'xanchor': 'center',
        'yanchor': 'top',
        'font_color': 'rgb(33, 107, 171)',
        'font_size': 30
    },
)

# 테이블 출력
fig.show()


### 2-2.3. 다음 단계를 위한 준비

- 미션 3-2 에서도 상위 100명의 응답자 데이터를 사용할 수 있도록 *top_100_rows* 데이터 프레임을 CSV 형태로 저장

In [ ]:
# top_100_rows를 Pickle 형식으로 저장
top_100_rows.to_pickle("/content/drive/MyDrive/top_100_rows.pkl")
print("top_100_rows가 '/content/drive/MyDrive/top_100_rows.pkl'에 저장되었습니다.")

top_100_rows가 '/content/drive/MyDrive/top_100_rows.pkl'에 저장되었습니다.
